In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import re

# set the graphs to show in the jupyter notebook
%matplotlib inline

# set seabor graphs to a better style
sns.set(style="ticks")

In [2]:
Customer = pd.read_csv('D:/python/proj_8/CUSTOMERS.csv')
geo_loc = pd.read_csv('D:/python/proj_8/GEO_LOCATION.csv')
ord_itm = pd.read_csv('D:/python/proj_8/ORDER_ITEMS.csv')
ord_pay = pd.read_csv('D:/python/proj_8/ORDER_PAYMENTS.csv')
ord_rev_rat = pd.read_csv('D:/python/proj_8/ORDER_REVIEW_RATINGS.csv')
orders = pd.read_csv('D:/python/proj_8/ORDERS.csv')
products = pd.read_csv('D:/python/proj_8/PRODUCTS.csv')
sellers = pd.read_csv('D:/python/proj_8/SELLERS.csv')

## Question 1:- Perform detailed exploratory analysis

**a**:-Define and calculate high level metrics like(Total revenue, Total quantity, Total products, Total categories, Total sellers, Total locations, Total channels, Total payment methods etc)

In [3]:
Total_revenue = ord_pay['payment_value'].sum()
Total_quantity = orders['order_id'].count()
Total_products = products['product_id'].count()
Total_sellers = sellers['seller_id'].count()
Total_location = geo_loc['geolocation_zip_code_prefix'].count()
Total_payment_methods = len(pd.unique(ord_pay['payment_type']))

**b**:-Understanding how many new customers acquired every month?

In [4]:
orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,10/2/2017 10:56,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,7/24/2018 20:41,7/26/2018 3:24,7/26/2018 14:31,8/7/2018 15:27,8/13/2018 0:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,8/8/2018 8:38,8/8/2018 8:55,8/8/2018 13:50,8/17/2018 18:06,9/4/2018 0:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,11/18/2017 19:28,11/18/2017 19:45,11/22/2017 13:39,12/2/2017 0:28,12/15/2017 0:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2/13/2018 21:18,2/13/2018 22:20,2/14/2018 19:46,2/16/2018 18:17,2/26/2018 0:00


In [5]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [6]:
cust_mon = orders.groupby(orders['order_purchase_timestamp'].dt.month)[['customer_id']].count()

In [7]:
new_cust_by_month = cust_mon.diff(axis=0)

In [8]:
new_cust_by_month

,customer_id
order_purchase_timestamp,
1,NaN
2,439.0
3,1385.0
4,-550.0
5,1230.0
6,-1161.0
7,906.0
8,525.0
9,-6538.0


**c**:-Understand the retention of customers on month on month basis. 

In [10]:
cust_mon['Tot_cust_at_beginning'] = cust_mon['customer_id']

In [11]:
cust_mon['Tot_cust_at_beginning'] =cust_mon.shift(periods=1 , axis=0)

In [12]:
cust_mon['retention'] = (cust_mon['customer_id'] / cust_mon['Tot_cust_at_beginning']) * 100

In [13]:
cust_mon

,customer_id,Tot_cust_at_beginning,retention
order_purchase_timestamp,,,
1,8069,NaN,NaN
2,8508,8069.0,105.440575
3,9893,8508.0,116.278796
4,9343,9893.0,94.440513
5,10573,9343.0,113.164936
6,9412,10573.0,89.019200
7,10318,9412.0,109.626009
8,10843,10318.0,105.088195
9,4305,10843.0,39.703034


**d**:-How the revenues from existing\new customers on month on month basis. 

In [14]:
orders.shape

(99441, 8)

In [15]:
ord_pay.shape

(103886, 5)

In [16]:
orders_paymt = pd.merge(left=orders , right=ord_pay , how='left' , left_on='order_id' , right_on='order_id')

In [17]:
orders_paymt.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,3.0,voucher,1.0,2.00


In [18]:
revenue_by_month = orders_paymt.groupby([orders_paymt['order_purchase_timestamp'].dt.month])[['payment_value']].sum()

In [19]:
revenue_by_month

,payment_value
order_purchase_timestamp,
1,1253492.22
2,1284371.35
3,1609515.72
4,1578573.51
5,1746900.97
6,1535156.88
7,1658923.67
8,1696821.64
9,732454.23


**e**:-

**f**:-Popular products by month,seller,state,category?

In [23]:
op = pd.merge(left=ord_itm , right=products , how='left' , left_on='product_id' , right_on='product_id')

In [27]:
orders_itm_products_st.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,1.0,87285b34884572647811a353c7ac498a,...,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,Chhuikhadan,Chhattisgarh


In [25]:
orders_itm_products = pd.merge(left=orders , right=op , how='left' , left_on='order_id' , right_on='order_id')

In [26]:
orders_itm_products_st = pd.merge(left=orders_itm_products , right=sellers , how='left' , left_on='seller_id' , right_on='seller_id')

In [32]:
orders_itm_products_st.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,1.0,87285b34884572647811a353c7ac498a,...,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,Chhuikhadan,Chhattisgarh


- Popular product by month:-->

In [41]:
prod_mon = orders_itm_products_st.groupby([orders_itm_products_st['order_purchase_timestamp'].dt.month , 'product_id'])[['product_id']].count().add_prefix('count_of_').reset_index()

In [29]:
prod_mon_max = prod_mon.groupby(['order_purchase_timestamp']).max()

In [30]:
prod_mon_max

,product_id,count_of_product_id
order_purchase_timestamp,,
1,ffe0fc4e02c3559643ac063fa5cf9d07,122
2,ffeb228c521d5464d1f71444da96c446,71
3,ffd4bf4306745865e5692f69bd237893,65
4,fffdb2d0ec8d6a61f0a0a0db3f25b441,85
5,fffe9eeff12fcbd74a2f2b007dde0c58,114
6,ffef256879dbadcab7e77950f4f4a195,74
7,fff81cc3158d2725c0655ab9ba0f712c,70
8,fffdb2d0ec8d6a61f0a0a0db3f25b441,68
9,fff6177642830a9a94a0f2cba5e476d1,37


- Popular product by seller:-->

In [43]:
prod_sell = orders_itm_products_st.groupby(['seller_id' , 'product_id'])[['product_id']].count().add_prefix('count_of_').reset_index()

In [44]:
prod_sell

,seller_id,product_id,count_of_product_id
0,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,3
1,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,113
2,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,17
3,001cca7ae9ae17fb1caed9dfb1094831,21fecd254a3103704126b28478ea7980,3
4,001cca7ae9ae17fb1caed9dfb1094831,4d7fee7877228c1497477ae53d97c214,2
...,...,...,...
34443,ffff564a4f9085cd26170f4732393726,cb17b798ba889575385281b7ec2fc22b,1
34444,ffff564a4f9085cd26170f4732393726,dbd024d4182504993ad1e3cd2ee9d9e9,2
34445,ffff564a4f9085cd26170f4732393726,de6517dda8e49774f58c07f80abc8d7a,1
34446,ffff564a4f9085cd26170f4732393726,e20b58fe57d487f33247e6cc1154eb9c,1


In [51]:
prod_by_seller = prod_sell.groupby(['seller_id' , 'product_id'])[['count_of_product_id']].max().sort_values(by = 'count_of_product_id' , ascending=False)

In [52]:
prod_by_seller

count_of_product_id
seller_id                        product_id                                           
955fee9216a65b617aa5c0531780ce60 aca2eb7d00ea1a7b8ebd4e68314663af                  527
1f50f920176fa81dab994f9023523100 422879e10f46682990de24d770e7f83d                  484
4a3ca9315b744ce9f8e9374361493884 99a4788cb24856965c36a24e339b6058                  482
1f50f920176fa81dab994f9023523100 389d119b48cf3043d311335e499d9c6b                  392
                                 368c6c730842d78016ad823897a372db                  388
...                                                                                ...
6edacfd9f9074789dad6d62ba7950b9c 759be9d8dec6d1622e7b0b8be27740fe                    1
                                 753d279facc7d9ca90f2e8bfb2bab052                    1
                                 73c32b5af96fecd16fed3efde88ea2c6                    1
                                 6feea907f9f949ce09d7d8ed626fd89a                    1
ffff564a4f9085cd26170f4732393726 fa2de9e6a52f697fbd74ddbc2c15b16c                    1

[34448 rows x 1 columns]

- Popular product by category:-->

In [47]:
prod_cat = orders_itm_products_st.groupby(['product_category_name' , 'product_id'])[['product_id']].count().add_prefix('count_of_').reset_index()

In [48]:
prod_cat

,product_category_name,product_id,count_of_product_id
0,Agro_Industry_And_Commerce,018ca97302e4293050cc41413194bb19,1
1,Agro_Industry_And_Commerce,026f43af35e7951067097527d5c31bcc,2
2,Agro_Industry_And_Commerce,07f01b6fcacc1b187a71e5074199db2d,1
3,Agro_Industry_And_Commerce,0a0adf0de1769b2970f3096d0b5abfb7,1
4,Agro_Industry_And_Commerce,0a27862bbf658a5b8cf655761bc5c72d,1
...,...,...,...
32323,Watches_Gifts,ff5d87897ed26d564711df324b98ebee,6
32324,Watches_Gifts,ff7b4560eaf93a8848cd5fde641f39c6,1
32325,Watches_Gifts,ff80f0538290775336ee74b4997a855c,1
32326,Watches_Gifts,ffd2365fb8224dc66883df9351d65deb,3


In [55]:
pop_products = prod_cat.groupby('product_category_name').max().sort_values(by = 'count_of_product_id' , ascending=False)

In [56]:
pop_products

,product_id,count_of_product_id
product_category_name,,
Furniture_Decor,ffbe169d395060d7fb975c990581a329,527
Bed_Bath_Table,ffe8083298f95571b4a66bfbc1c05524,488
Garden_Tools,ff8e9a180c931adab01ed6b3c44edc25,484
Computers_Accessories,fffdb2d0ec8d6a61f0a0a0db3f25b441,343
Watches_Gifts,fff515ea94dbf35d54d256b3e39f0fea,323
...,...,...
Dvds_Blu_Ray,fe1ba35c47a4b599f34ce8ad2aa2c42b,5
Music,fac21a1c91c8001793435c475fa7f032,4
Fashion_Childrens_Clothes,8cfc3506cedc0626364457d254429118,3


- Popular product by state:-->

In [63]:
Customer.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [67]:
orders_itm_products.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object')

In [68]:
prod_st = pd.merge(left=orders_itm_products , right=Customer , how='left' , left_on='customer_id' , right_on='customer_id')

In [70]:
prod_st_cnt = prod_st.groupby(['customer_state' , 'product_id'])[['product_id']].count().add_prefix('count_of_').reset_index()

In [72]:
pop_prod_st = prod_st_cnt.groupby(['customer_state']).max()

In [73]:
pop_prod_st

,product_id,count_of_product_id
customer_state,,
Andhra Pradesh,fffdb2d0ec8d6a61f0a0a0db3f25b441,345
Arunachal Pradesh,ff8b5a5f01bf21afd7d49b4688ca6d1f,12
Chhattisgarh,fffdb2d0ec8d6a61f0a0a0db3f25b441,40
Delhi,ffd60d515c690a976c497e75cd2336f4,22
Goa,dd231637766e756fd1cf2fd80501fce1,1
Gujarat,fffe9eeff12fcbd74a2f2b007dde0c58,45
Haryana,ffd4bf4306745865e5692f69bd237893,13
Himachal Pradesh,ff5f8606556b625613f069a5a9a48966,6
Jammu & Kashmir,ffc9caf33e2d1e9f44e3e06da19085f7,15


 ### (g) Popular categories by state and month

In [74]:
orpr = pd.merge(left=products , right=ord_itm , how='left' , left_on='product_id' , right_on='product_id')

In [75]:
orcu = pd.merge(left=orders , right=Customer , how='left' , left_on='customer_id' , right_on='customer_id')

In [76]:
cat_st_mon = pd.merge(left=orpr , right=orcu , how='left' , left_on='order_id' , right_on='order_id')

- Popular categories by state:-->

In [77]:
pop_cat_by_state = cat_st_mon.groupby(['customer_state' , 'product_category_name'])[['product_category_name']].count().add_prefix('count_of_').reset_index()

In [78]:
category_st = pop_cat_by_state.groupby('customer_state').max()

In [79]:
category_st

,product_category_name,count_of_product_category_name
customer_state,,
Andhra Pradesh,Watches_Gifts,7101
Arunachal Pradesh,Watches_Gifts,114
Chhattisgarh,Watches_Gifts,681
Delhi,Watches_Gifts,328
Goa,Telephony,1
Gujarat,Watches_Gifts,786
Haryana,Watches_Gifts,282
Himachal Pradesh,Watches_Gifts,83
Jammu & Kashmir,Watches_Gifts,160


- Popular categories by month:-->

In [80]:
category_month = cat_st_mon.groupby([cat_st_mon['order_purchase_timestamp'].dt.month , 'product_category_name'])[['product_category_name']].count().add_prefix('count_of_').reset_index()

In [81]:
category_month_max = category_month.groupby('order_purchase_timestamp').max()

In [82]:
category_month_max

,product_category_name,count_of_product_category_name
order_purchase_timestamp,,
1,Watches_Gifts,896
2,Watches_Gifts,1087
3,Watches_Gifts,1087
4,Watches_Gifts,1020
5,Watches_Gifts,1116
6,Watches_Gifts,1155
7,Watches_Gifts,1203
8,Watches_Gifts,1209
9,Watches_Gifts,533


 ### (h) List top 10 most expensive products sorted by price.

In [83]:
ord_itm.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,9/19/2017 9:45,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,5/3/2017 11:05,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1/18/2018 14:48,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,8/15/2018 10:10,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2/13/2017 13:57,199.90,18.14


In [84]:
top_10_Most_expensive = ord_itm.loc[: , ['product_id' , 'price']].sort_values(by = 'price' , ascending = False)

In [85]:
top_10_Most_expensive.head(10)

,product_id,price
3556,489ae2aa008f021502940f251d4cce7f,6735.00
112233,69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
107841,1bdf5e6731585cf01aa8169c7028d6ad,6499.00
74336,a6492cc69376c469ab6f61d8f44de961,4799.00
11249,c3ed642d592594bb648ff4a04cee2747,4690.00
62086,259037a6a41845e455183f89c5035f18,4590.00
29193,a1beef8f3992dbd4cd8726796aa69c53,4399.87
45843,6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
78310,dd113cb02b2af9c8e5787e8f1f0722f6,4059.00
59137,6902c1962dd19d540807d0ab8fade5c6,3999.90


## Question 2:- Performing customers and sellers segmentation.

 ### (a) Divide the customers into groups based on the revenue generated.

In [86]:
from numpy import where as IF

In [87]:
cust_group = IF((ord_pay['payment_value'] > 0) & (ord_pay['payment_value'] <= 150) , 'Bronze' ,
               IF((ord_pay['payment_value'] > 150) & (ord_pay['payment_value'] <=300) , 'Silver' , 
                 IF((ord_pay['payment_value'] > 300) & (ord_pay['payment_value'] <= 450) , 'Gold' , 'Premium')))

In [88]:
cust_group = pd.Series(cust_group , name = 'cust_group')

In [89]:
cust_group

0         Bronze
1         Bronze
2         Bronze
3         Bronze
4         Bronze
           ...  
103881      Gold
103882    Bronze
103883    Bronze
103884      Gold
103885    Silver
Name: cust_group, Length: 103886, dtype: object

In [90]:
revenue = ord_pay['payment_value'] 

In [91]:
revenue = pd.Series(revenue , name='revenue')

In [92]:
customer = Customer['customer_id']

In [93]:
cust_group_revenue = pd.concat([customer , revenue , cust_group] , axis=1)

In [94]:
cust_segmentation = cust_group_revenue.groupby(['customer_id' , 'cust_group'])[['revenue']].sum().reset_index()

In [95]:
cust_group_revenue

,customer_id,revenue,cust_group
0,06b8999e2fba1a1fbc88172c00ba8bc7,99.33,Bronze
1,18955e83d337fd6b2def6b18a428ac77,24.39,Bronze
2,4e7b3e00288586ebd08712fdd0374a03,65.71,Bronze
3,b2b6027bc5c5109e529d4dc6358b12c3,107.78,Bronze
4,4f2d8ab171c80ec8364f7c12e35b23ad,128.45,Bronze
...,...,...,...
103881,NaN,363.31,Gold
103882,NaN,96.80,Bronze
103883,NaN,47.77,Bronze
103884,NaN,369.54,Gold


 ### (b) Divide the sellers into groups based on the revenue generated.

In [96]:
seller_group = IF((ord_pay['payment_value'] > 0) & (ord_pay['payment_value'] <= 400) , 'avg_seller' ,
                 IF((ord_pay['payment_value'] > 400) & (ord_pay['payment_value'] <= 800) , 'good_seller' ,
                   IF((ord_pay['payment_value'] > 800) & (ord_pay['payment_value'] <= 1200) , 'Excellent_seller' , 'Premium_seller')))

In [97]:
seller_group = pd.Series(seller_group , name='seller_group')

In [98]:
seller_revenue = ord_pay['payment_value'] 

In [99]:
seller_revenue = pd.Series(seller_revenue , name='seller_revenue')

In [100]:
Seller = sellers.seller_id

In [101]:
all_seller_group = pd.concat([Seller , seller_revenue , seller_group] , axis=1)

In [102]:
seller_segmentation = all_seller_group.groupby(['seller_id' , 'seller_group'])[['seller_revenue']].sum().reset_index()

In [103]:
seller_segmentation

,seller_id,seller_group,seller_revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,avg_seller,318.87
1,001cca7ae9ae17fb1caed9dfb1094831,avg_seller,41.53
2,001e6ad469a905060d959994f1b41e4f,avg_seller,12.29
3,002100f778ceb8431b7a1020ff7ab48f,avg_seller,35.40
4,003554e2dce176b5555353e4f3555ac8,avg_seller,195.43
...,...,...,...
3090,ffcfefa19b08742c5d315f2791395ee5,avg_seller,125.53
3091,ffdd9f82b9a447f6f8d4b91554cc7dd3,avg_seller,198.50
3092,ffeee66ac5d5a62fe688b9d26f83f534,avg_seller,135.60
3093,fffd5413c0700ac820c7069d66d98c89,avg_seller,141.47


## Question 3:- Cross-Selling(Which Products are Selling together)

In [104]:
cat_st_mon.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_id', 'order_item_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state'],
      dtype='object')

In [105]:
ord_prod = cat_st_mon.groupby(['order_id' , 'product_id'])[['product_id']].count().add_prefix('count_of_').reset_index()

In [106]:
Top_10_combinations = ord_prod.loc[ord_prod['count_of_product_id'] > 1 , :].sort_values(by = 'count_of_product_id' , ascending=False).head(10)

In [107]:
Top_10_combinations

,order_id,product_id,count_of_product_id
10850,1b15974a0141d54e36626dca3fdc731a,ee3d532c8a438679776d222e997606b3,20
68186,ab14fdcfbe524636d65ee38360e22ce8,9571759451b1d780ee7c15012ea109d4,20
63242,9ef13efd6949e4573a18964dd1bbe7f5,37eb69aca8718e843d897aa7b82f462d,15
26550,428a2f660dc84138d969ccd69a0ab6d5,89b190a046022486c635022524a974a8,15
46268,73c8ab38f07dc94389065f7eba4f297a,422879e10f46682990de24d770e7f83d,14
61929,9bdc4d4c71aa1de4606060929dee888c,44a5d24dd383324a421569ca697b13c2,14
22288,37ee401157a3a0b28c9c6d0ed8c3b24b,d34c07a2d817ac73f4caf8c574215fed,13
23217,3a213fcdfe7d98be74ea0dc05a8b31ae,a62e25e09e05e6faf31d90c6ec1aa3d1,12
17469,2c2a19b5703863c908512d135aa6accc,03e1c946c0ddfc58724ff262aef08dff,12
43243,6c355e2913545fa6f72c40cbca57729e,32e18e89237933ebdaaebd78a27e7fa1,11


## Question 4:- Payment Behaviour

 ### (a) How customers are paying?

In [108]:
orcust = pd.merge(left = orders , right = Customer , how = 'left' , left_on='customer_id' , right_on='customer_id')

In [109]:
ord_cust_pay = pd.merge(left = orcust , right = ord_pay , how = 'left' , left_on='order_id' , right_on='order_id')

In [110]:
ord_cust_pay.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,7c396fd4830fd04220f754e42b4e5bff,3149,Akkarampalle,Andhra Pradesh,1.0,credit_card,1.0,18.12


In [111]:
cust_payment_type = pd.DataFrame(pd.unique(ord_cust_pay['payment_type']))

In [112]:
cust_payment_type

,0
0,credit_card
1,voucher
2,UPI
3,debit_card
4,not_defined
5,NaN


 ### (b) Which payment channels are used by most customers?

In [113]:
pay_type_by_most_cust = ord_pay.groupby('payment_type')[['payment_type']].count().add_prefix('count_of_').sort_values(by = 'count_of_payment_type' , ascending=False)

In [114]:
pay_type_by_most_cust

,count_of_payment_type
payment_type,
credit_card,76795
UPI,19784
voucher,5775
debit_card,1529
not_defined,3


## Question 5:- Customer satisfaction towards category & product

 ### (a) Which categories(top 10) are maximum rated and minimum rated?

#### Maximum

In [115]:
ord_rev_all = pd.merge(left=orders_itm_products , right=ord_rev_rat , how='left' , left_on='order_id' , right_on='order_id')

In [118]:
ord_rev_all.groupby(['product_category_name'])[['review_score']].sum().add_prefix('sum_of_').sort_values(by = 'sum_of_review_score' , ascending = False).head(10)

,sum_of_review_score
product_category_name,
Bed_Bath_Table,43636
Health_Beauty,40121
Sports_Leisure,35616
Furniture_Decor,32716
Computers_Accessories,30953
Housewares,28235
Watches_Gifts,24016
Telephony,17907
Garden_Tools,17572


#### Minimum

In [119]:
ord_rev_all.groupby(['product_category_name'])[['review_score']].sum().add_prefix('sum_of_').sort_values(by = 'sum_of_review_score' , ascending = True).head(10)

,sum_of_review_score
product_category_name,
Security_And_Services,5
Fashion_Childrens_Clothes,36
La_Cuisine,53
Cds_Dvds_Musicals,65
Arts_And_Craftmanship,99
Home_Comfort_2,101
Diapers_And_Hygiene,127
Fashion_Sport,132
Flowers,139


 ### (b) Which products(top 10) are maximum rated and minimum rated?

#### Maximum

In [120]:
ord_rev_all.groupby('product_id')[['review_score']].sum().add_prefix('sum_of_').sort_values(by = 'sum_of_review_score' , ascending = False).head(10)

,sum_of_review_score
product_id,
aca2eb7d00ea1a7b8ebd4e68314663af,2112
422879e10f46682990de24d770e7f83d,1920
99a4788cb24856965c36a24e339b6058,1896
389d119b48cf3043d311335e499d9c6b,1612
368c6c730842d78016ad823897a372db,1531
53759a2ecddad2bb87a079a1f1519f73,1448
d1c427060a0f73f6b889a5c7c61f2ac4,1432
53b36df67ebb7c41585e8d54d6772e08,1348
154e7e31ebfa092203795c972e5804a6,1264


#### Minimum

In [121]:
ord_rev_all.groupby('product_id')[['review_score']].sum().add_prefix('sum_of_').sort_values(by = 'sum_of_review_score' , ascending = True).head(10)

,sum_of_review_score
product_id,
e03071a2d2410c9ef2be47b508cac95f,1
b56ff361e0733eebc1d010432936b706,1
7ed8c74b7ad2f1790a24a24c071da537,1
43c81702c9aca96af077286a5b2525b9,1
1da4613d4f7e10e1554538b46dd16486,1
ed9d81a5043f74cbe0e3db315770af45,1
5096e827f6152df3974343e735a52dc9,1
1da0fbb35cfdc9fcb6d48dfddca69513,1
1d9c9e0532fb481c0f89c3b20cec62e6,1


 ### (c) Average rating by location,seller,category,month etc.

In [129]:
Customer.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [128]:
ord_rev_all.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'review_id', 'review_score',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object')

In [130]:
ord_rev_loc = pd.merge(left=ord_rev_all , right=Customer , how='left' , left_on='customer_id' , right_on='customer_id')

#### Location

In [135]:
ord_rev_loc.groupby('customer_state')[['review_score']].mean().add_prefix('Avg_of_').reset_index()

,customer_state,Avg_of_review_score
0,Andhra Pradesh,3.980414
1,Arunachal Pradesh,4.024219
2,Chhattisgarh,4.015419
3,Delhi,4.023483
4,Goa,4.800000
5,Gujarat,4.018021
6,Haryana,4.080142
7,Himachal Pradesh,3.984706
8,Jammu & Kashmir,4.006211
9,Karnataka,3.996825


#### Seller

In [136]:
ord_rev_loc.groupby('seller_id')[['review_score']].mean().add_prefix('Avg_of_').reset_index()

,seller_id,Avg_of_review_score
0,0015a82c2db000af6aaaf3ae2ecb0532,3.666667
1,001cca7ae9ae17fb1caed9dfb1094831,3.874477
2,001e6ad469a905060d959994f1b41e4f,1.000000
3,002100f778ceb8431b7a1020ff7ab48f,3.982143
4,003554e2dce176b5555353e4f3555ac8,5.000000
...,...,...
3090,ffcfefa19b08742c5d315f2791395ee5,1.000000
3091,ffdd9f82b9a447f6f8d4b91554cc7dd3,4.250000
3092,ffeee66ac5d5a62fe688b9d26f83f534,4.214286
3093,fffd5413c0700ac820c7069d66d98c89,3.885246


#### Category

In [137]:
ord_rev_loc.groupby('product_category_name')[['review_score']].mean().add_prefix('Avg_of_').reset_index()

,product_category_name,Avg_of_review_score
0,Agro_Industry_And_Commerce,4.000000
1,Air_Conditioning,3.919192
2,Art,3.918660
3,Arts_And_Craftmanship,4.125000
4,Audio,3.808219
...,...,...
66,Stationery,4.181386
67,Tablets_Printing_Image,4.048193
68,Telephony,3.935604
69,Toys,4.138667


#### Month

In [139]:
ord_rev_loc.groupby(ord_rev_loc['order_purchase_timestamp'].dt.month)[['review_score']].mean().add_prefix('Avg_of_')

,Avg_of_review_score
order_purchase_timestamp,
1,3.930348
2,3.766168
3,3.763856
4,4.038792
5,4.107498
6,4.148724
7,4.162470
8,4.185657
9,4.091259
